# Outline

Section 1: [Imports](#imports)
* [Import Libraries](#libraries)
* [Default Preferences](#preferences)
* [Import Data](#data) <br>

Section 2: [Data Wrangling](#wrangling)
* [Remove Cancelled Orders](#cancelled)
* [Crease 'Sales'](#sales)
* [Basic Exploration](#basic)
* [Numeric Anomolies](#num_anomolies)
    * [Negative Price](#neg_price)
    * [Negative Quantity](#neg_quant)
    * [Price == 0](#zero_price)
    * [Excessively High Price](#high_price)
    * [Adjustments](#adjust)
* [Unique Values](#unique)
    * [StockCode](#stockcode)
    * [Description](#description)
    * [StockCode with Multiple Descriptions](#mismatch)
    * [Customer ID](#cust_id)
    * [Country](#country)
* [Missing Values](#missing)

Section 3: [Export Clean Data & Follow-Up Points](#end)
* [Export Clean Data](#export)
* [Follow-Up Points for EDA](#explore)

# Section 1: Imports<a id="imports"></a>


<a id= 'libraries'></a>
**Import Libraries** 

In [9]:
import pandas as pd
import numpy as np
import time

<a id='preferences'></a>
**Set Default Preferences**

In [11]:
# display all columns
pd.set_option('display.max_columns', None)

# surpress scientific notation
pd.options.display.float_format = '{:}'.format

<a id='data'></a>
**Import Data** 
<br>
Data was found in the UC Irvine Machine Learning Repository.  Dataset can be found [here](https://archive.ics.uci.edu/dataset/502/online+retail+ii).

Data is in file name 'online_retail_II.xlsx' and spread across two sheets: 'Year 2009-2010', 'Year 2010-2011'.  Data should have 1,067,371 rows and 8 columns.

In [13]:
time_text = f'Time to execute (seconds):'

In [14]:
start = time.time()

# import
file_path = '../Data/online_retail_II.xlsx'
sheet_0 = pd.read_excel(file_path, sheet_name=0)
sheet_1 = pd.read_excel(file_path, sheet_name=1)

# combine sheets to df
raw_data = pd.concat([sheet_0, sheet_1], ignore_index=True)

end = time.time()

print(time_text, end-start)

Time to execute (seconds): 41.616738080978394


In [15]:
# sanity check
print('DataFrame has {} rows and {} columns'.format(raw_data.shape[0],raw_data.shape[1]))

DataFrame has 1067371 rows and 8 columns


In [16]:
# create copy for if/when something goes wrong so I dont have to re-import data
df = raw_data.copy(deep=True)

# Section 2: Data Wrangling <a id='wrangling'></a>
<br>
The data represents online sales data for an UK based company from 01/12/2009 and 09/12/2011. Per the dataset description, the company sells 'unique all-occasion gift-ware' and many of the customers are wholesalers.

Description of columns provided by data owner:<br>

* InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation. 
* StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product. 
* Description: Product (item) name. Nominal. 
* Quantity: The quantities of each product (item) per transaction. Numeric.	
* InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated. 
* UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£). 
* CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer. 
* Country: Country name. Nominal. The name of the country where a customer resides.

## Remove Cancelled orders<a id='cancelled'></a>
Per column descriptions, orders that were cancelled have a 'c' at the beginning of the invoice number.  This analysis is not interested in cancelled orders.

In [19]:
#df[df['Invoice'].notna() & df['Invoice'].str.startswith('c')] <-- no entries
df[df['Invoice'].notna() & df['Invoice'].str.startswith('C')]

# cancelled orders acrually start with capital 'C'

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.1,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [20]:
# save cancelled orders to seperate dataframe
cancelled_orders = df[df['Invoice'].notna() & df['Invoice'].str.startswith('C')]
cancelled_orders.shape

(19494, 8)

In [21]:
# drop cancelled orders from df
df = df[~df.Invoice.isin(cancelled_orders.Invoice)]
df.shape

(1047877, 8)

In [22]:
#check
#len(raw_data)-len(cancelled_orders) #1047877

## Create Sales <a id='sales'></a>
Use Quantity times Price to create Sales

In [24]:
df['Sales'] = df['Quantity'] * df['Price']

## Basic Exploration<a id='basic'></a>

In [26]:
df.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.1,13085.0,United Kingdom,100.80000000000001
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom,39.599999999999994
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom,59.5
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom,30.599999999999998
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom,45.0


In [27]:
df.sample(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
701160,551982,84792,ENCHANTED BIRD COATHANGER 5 HOOK,2,2011-05-05 13:56:00,4.65,15023.0,United Kingdom,9.3
792718,560292,20712,JUMBO BAG WOODLAND ANIMALS,1,2011-07-18 10:19:00,4.13,NaN,United Kingdom,4.13
427129,530150,35643,?,-223,2010-11-01 16:53:00,0.0,NaN,United Kingdom,-0.0
888364,568528,23200,JUMBO BAG PEARS,10,2011-09-27 13:32:00,2.08,13979.0,United Kingdom,20.8
1008,489546,82494L,WOODEN FRAME ANTIQUE WHITE,6,2009-12-01 12:30:00,2.95,14156.0,EIRE,17.700000000000003
662539,548012,22900,SET 2 TEA TOWELS I LOVE LONDON,20,2011-03-29 11:15:00,2.95,17377.0,United Kingdom,59.0
501465,536174,21527,RED RETROSPOT TRADITIONAL TEAPOT,6,2010-11-30 12:14:00,7.95,17428.0,United Kingdom,47.7
32049,491971,11001,ASSTD DESIGN RACING CAR PEN,1,2009-12-14 18:37:00,3.43,NaN,United Kingdom,3.43
357771,524112,22083,PAPER CHAIN KIT RETRO SPOT,6,2010-09-27 13:24:00,2.95,15358.0,United Kingdom,17.700000000000003
587783,541497,84251C,"GREETING CARD, TWO SISTERS.",2,2011-01-18 15:19:00,0.42,NaN,United Kingdom,0.84


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1047877 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1047877 non-null  object        
 1   StockCode    1047877 non-null  object        
 2   Description  1043495 non-null  object        
 3   Quantity     1047877 non-null  int64         
 4   InvoiceDate  1047877 non-null  datetime64[ns]
 5   Price        1047877 non-null  float64       
 6   Customer ID  805620 non-null   float64       
 7   Country      1047877 non-null  object        
 8   Sales        1047877 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 79.9+ MB


In [29]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID,Sales
count,1047877.0,1047877,1047877.0,805620.0,1047877.0
mean,10.592353873593943,2011-01-03 07:10:41.287592704,3.901109145443598,15331.85624984484,19.862940429077078
min,-9600.0,2009-12-01 07:45:00,-53594.36,12346.0,-53594.36
25%,1.0,2010-07-09 15:48:00,1.25,13982.0,3.78
50%,3.0,2010-12-07 15:28:00,2.1,15271.0,9.95
75%,10.0,2011-07-22 15:02:00,4.13,16805.0,17.700000000000003
max,80995.0,2011-12-09 12:50:00,25111.09,18287.0,168469.6
std,135.28060251167264,NaN,94.35774019466297,1696.7683953203334,217.4654739371106


## Numeric Anomolies <a id='num_anomolies'></a>

**Negative Price / Bad debt**<br> <a id='neg_price'></a>
All negative price items are bad debt expenses and not sales. They will be dropped. There is one additional bad debt expense that has a positive Price that will be dropped as well.

In [32]:
df[df.Price<0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom,-53594.36
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom,-44031.79
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom,-38925.87
825444,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,-11062.06
825445,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,-11062.06


In [33]:
#check for other bad debt expenses
df.loc[df.Description=='Adjust bad debt']

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom,-53594.36
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom,-44031.79
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom,-38925.87
825443,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom,11062.06
825444,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,-11062.06
825445,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,-11062.06


The positive bad debt looks like it was added as a positive accidentlly and then corrected with two negatives right after.

In [35]:
# drop bad debt transactions
df=df[df.Description!='Adjust bad debt']


df.describe()

,Quantity,InvoiceDate,Price,Customer ID,Sales
count,1047871.0,1047871,1047871.0,805620.0,1047871.0
mean,10.592408798411254,2011-01-03 07:10:27.055716096,4.042001952530419,15331.85624984484,20.00392463194421
min,-9600.0,2009-12-01 07:45:00,0.0,12346.0,-0.0
25%,1.0,2010-07-09 15:48:00,1.25,13982.0,3.78
50%,3.0,2010-12-07 15:28:00,2.1,15271.0,9.95
75%,10.0,2011-07-22 15:02:00,4.13,16805.0,17.700000000000003
max,80995.0,2011-12-09 12:50:00,25111.09,18287.0,168469.6
std,135.28098786633166,NaN,50.145884231710276,1696.7683953203334,202.23298303464352


**Negative Quantity** <br> <a id='neg_quant'></a>
The 'Descriptions' from negative makes it seem these are a mix of returns from damages, mixups, errors, short stock and other issues.

In [37]:
df[df.Quantity < 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.0,NaN,United Kingdom,-0.0
283,489463,71477,short,-240,2009-12-01 10:52:00,0.0,NaN,United Kingdom,-0.0
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.0,NaN,United Kingdom,-0.0
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom,-0.0
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom,-0.0
...,...,...,...,...,...,...,...,...,...
1060794,581210,23395,check,-26,2011-12-07 18:36:00,0.0,NaN,United Kingdom,-0.0
1060796,581212,22578,lost,-1050,2011-12-07 18:38:00,0.0,NaN,United Kingdom,-0.0
1060797,581213,22576,check,-30,2011-12-07 18:38:00,0.0,NaN,United Kingdom,-0.0
1062371,581226,23090,missing,-338,2011-12-08 09:56:00,0.0,NaN,United Kingdom,-0.0


In [38]:
# all values with quantity < 0 have a Price == 0 
all(df.loc[df.Quantity<0,'Price']==0)

True

In [39]:
df[(df.Quantity < 0) &  (df.Description.isna())]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom,-0.0
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom,-0.0
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom,-0.0
4566,489821,85049G,NaN,-240,2009-12-02 13:25:00,0.0,NaN,United Kingdom,-0.0
6576,489901,21098,NaN,-200,2009-12-03 09:47:00,0.0,NaN,United Kingdom,-0.0
...,...,...,...,...,...,...,...,...,...
1043180,580087,84341B,NaN,-2,2011-12-01 14:27:00,0.0,NaN,United Kingdom,-0.0
1046794,580359,20775,NaN,-35,2011-12-02 16:11:00,0.0,NaN,United Kingdom,-0.0
1047621,580379,72225C,NaN,-144,2011-12-02 17:57:00,0.0,NaN,United Kingdom,-0.0
1047623,580381,21758,NaN,-9,2011-12-02 17:58:00,0.0,NaN,United Kingdom,-0.0


In [40]:
df[(df.Quantity < 0) &  (df.Description.isna())].shape[0]/len(df[df.Quantity < 0])

0.7778420595892392

In [41]:
df[df.Quantity < 0]['Description'].value_counts().head(20)

Description
check                     123
damages                    84
?                          83
damaged                    78
missing                    27
sold as set on dotcom      20
Damaged                    17
thrown away                 9
smashed                     9
Unsaleable, destroyed.      9
dotcom                      8
??                          7
damages?                    7
crushed                     6
given away                  6
counted                     5
wet damaged                 5
checked                     5
MIA                         5
Damages                     5
Name: count, dtype: int64

In [42]:
df[df.Quantity < 0]['Description'].unique()

array(['85123a mixed', 'short', '21733 mixed', nan, 'lost', 'damages',
       'invcd as 84879?', 'sold as gold', 21494, 'lost?', 'damaged',
       'wet', 'MIA', 'smashed', 'bad quality', 'discoloured', 'missing',
       'missing (wrongly coded?)', 'Damages', 'Dotcom multiples',
       'damaged?', 'damages?', 'No Stock', 'my error - connor', 'Dotcom',
       '?', 'wedding co returns?', 'damages, lost bits etc',
       'lost in space', 'ebay sales', 'MISSING', "12'S SOLD AS 1?",
       'missing?', 'gone', 'dirty', 'Damp and rusty', 'invoice 506647',
       'Amazon sold as set', 'Dotcom sold as set', '17129c',
       'phil said so', 'wet and rotting', 'wet & rotting', 'display',
       'wrong ctn size', 'sold as 17003?', 'Zebra invcing error',
       'damages etc', 'Damaged', 'sold in wrong qnty', 'dotcom sales',
       'Rusty ', 'entry error', 'damaged/dirty', '?????', 'dotcom',
       'donated to the Food Chain charity.',
       'Donated to The Food Chain charity', 'Wet, rusty-thrown aw

In [43]:
df[df.Quantity==0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales


78% of transactions with a negative quantity do no have a description. However, the transactions that do have Descriptions makes it seem these are returns from a mix of issues like damages, mixups, errors, short stock, etc.  So they will be dropped.

In [45]:
# include only non necative Quantity
df=df[df.Quantity>0]

In [46]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID,Sales
count,1044414.0,1044414,1044414.0,805620.0,1044414.0
mean,11.176183965362394,2011-01-03 13:08:49.350601984,4.055380938976305,15331.85624984484,20.07013742443131
min,1.0,2009-12-01 07:45:00,0.0,12346.0,0.0
25%,1.0,2010-07-11 14:19:00,1.25,13982.0,3.9
50%,3.0,2010-12-07 15:32:00,2.1,15271.0,9.959999999999999
75%,10.0,2011-07-24 11:25:00,4.13,16805.0,17.700000000000003
max,80995.0,2011-12-09 12:50:00,25111.09,18287.0,168469.6
std,129.45647979052785,NaN,50.228266838967045,1696.7683953203334,202.56412126680448


<a id='zero_price'></a>
**Price == 0**

In [48]:
df[df.Price==0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom,0.0
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,NaN,United Kingdom,0.0
4674,489825,22076,6 RIBBONS EMPIRE,12,2009-12-02 13:34:00,0.0,16126.0,United Kingdom,0.0
5904,489861,DOT,DOTCOM POSTAGE,1,2009-12-02 14:50:00,0.0,NaN,United Kingdom,0.0
6378,489882,35751C,NaN,12,2009-12-02 16:22:00,0.0,NaN,United Kingdom,0.0
...,...,...,...,...,...,...,...,...,...
1060795,581211,22142,check,14,2011-12-07 18:36:00,0.0,NaN,United Kingdom,0.0
1062442,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom,0.0
1063965,581406,46000M,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,0.0,NaN,United Kingdom,0.0
1063966,581406,46000S,POLYESTER FILLER PAD 40x40cm,300,2011-12-08 13:58:00,0.0,NaN,United Kingdom,0.0


In [49]:
df[df.Price==0]['Description'].nunique()

399

In [50]:
df[df.Price==0]['Description'].value_counts()

Description
check                                39
found                                28
OWL DOORSTOP                         15
adjustment                           14
POLYESTER FILLER PAD 45x45cm         12
                                     ..
SILK PURSE RUSSIAN DOLL PINK          1
LOVE HEART TRINKET POT                1
S/6 WOODEN SKITTLES IN COTTON BAG     1
DOORMAT RED SPOT                      1
CLASSIC GLASS COOKIE JAR              1
Name: count, Length: 399, dtype: int64

In [51]:
df[df.Price==0]['Customer ID'].value_counts()

Customer ID
12647.0    6
14646.0    5
13081.0    4
13047.0    3
14911.0    2
14108.0    2
12415.0    2
13985.0    2
17667.0    2
12748.0    2
16133.0    1
14410.0    1
12457.0    1
16818.0    1
12507.0    1
15581.0    1
18059.0    1
15804.0    1
12446.0    1
13239.0    1
14110.0    1
12437.0    1
12431.0    1
13014.0    1
15602.0    1
12603.0    1
12444.0    1
16406.0    1
13113.0    1
16126.0    1
17560.0    1
12623.0    1
17231.0    1
15070.0    1
18071.0    1
14258.0    1
12417.0    1
16858.0    1
14103.0    1
14827.0    1
14045.0    1
15107.0    1
13089.0    1
14025.0    1
12471.0    1
17450.0    1
13554.0    1
12820.0    1
16560.0    1
15658.0    1
13256.0    1
Name: count, dtype: int64

In [52]:
df[df.Price==0]['Country'].value_counts()

Country
United Kingdom    2717
Germany              9
Netherlands          5
EIRE                 5
Australia            3
Belgium              1
Switzerland          1
Spain                1
RSA                  1
France               1
Norway               1
Name: count, dtype: int64

In [53]:
df[df.Price==0]['StockCode'].value_counts()

StockCode
21116     15
22501     15
79321     15
22734     13
23084     13
          ..
79067      1
22441      1
51020B     1
47598      1
85175      1
Name: count, Length: 1395, dtype: int64

There is no clear pattern to the data that has a Price == 0 other than the vast majority come from the UK.  The Descriptions suggest tht some of those transactions are adjustments to other transactions as well.

Given that a large portion of the company's customers are wholesalers it can be guessed that some of these transactions are also free samples.

This analysis is primarily looking to group customers by groups for marketing or other sales purposes.  As such, all transactions with a Price == 0 will be dropped.

In [55]:
# drop Price == 0
df = df[df.Price > 0]

In [56]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID,Sales
count,1041669.0,1041669,1041669.0,805549.0,1041669.0
mean,10.963467281833289,2011-01-03 16:31:35.975728384,4.066067654888452,15331.954969840444,20.123026132101465
min,1.0,2009-12-01 07:45:00,0.001,12346.0,0.001
25%,1.0,2010-07-12 10:26:00,1.25,13982.0,3.9
50%,3.0,2010-12-07 15:33:00,2.1,15271.0,9.96
75%,10.0,2011-07-24 12:05:00,4.13,16805.0,17.700000000000003
max,80995.0,2011-12-09 12:50:00,25111.09,18287.0,168469.6
std,126.51505066216843,NaN,50.29397198844494,1696.7370388955817,202.82822022343726


<a id='high_price'></a>
**Excessively High Price**

In [58]:
# look at top 1% prices
threshold = df.Price.quantile(0.99)
df[df.Price>threshold]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
126,489444,POST,POSTAGE,1,2009-12-01 09:55:00,141.0,12636.0,USA,141.0
173,489447,POST,POSTAGE,1,2009-12-01 10:10:00,130.0,12362.0,Belgium,130.0
881,489537,21737,SET 3 WICKER LOG BASKETS,1,2009-12-01 12:14:00,19.95,14040.0,United Kingdom,19.95
1344,489562,84472,PINK AND LILAC QUILTED THROW,1,2009-12-01 13:07:00,45.95,17998.0,United Kingdom,45.95
1381,489566,85067,CREAM SWEETHEART WALL CABINET,2,2009-12-01 13:16:00,18.95,17742.0,United Kingdom,37.9
...,...,...,...,...,...,...,...,...,...
1066898,581498,22622,BOX OF VINTAGE ALPHABET BLOCKS,1,2011-12-09 10:26:00,20.79,NaN,United Kingdom,20.79
1066916,581498,22776,SWEETHEART 3 TIER CAKE STAND,1,2011-12-09 10:26:00,19.96,NaN,United Kingdom,19.96
1066920,581498,22838,3 TIER CAKE TIN RED AND CREAM,1,2011-12-09 10:26:00,29.17,NaN,United Kingdom,29.17
1066988,581498,84968e,SET OF 16 VINTAGE BLACK CUTLERY,1,2011-12-09 10:26:00,24.96,NaN,United Kingdom,24.96


Most prices in the top 1% of prices are still reasonable for products or are postage.  

In [60]:
df[df['Price'] > 2000].sort_values(by='Price', ascending=False)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
241827,512771,M,Manual,1,2010-06-17 16:53:00,25111.09,NaN,United Kingdom,25111.09
517955,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom,13541.33
540478,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom,13541.33
135013,502263,M,Manual,1,2010-03-23 15:22:00,10953.5,12918.0,United Kingdom,10953.5
135015,502265,M,Manual,1,2010-03-23 15:28:00,10953.5,NaN,United Kingdom,10953.5
342147,522796,M,Manual,1,2010-09-16 15:12:00,10468.8,NaN,United Kingdom,10468.8
372834,525399,M,Manual,1,2010-10-05 11:49:00,10468.8,NaN,United Kingdom,10468.8
358639,524159,M,Manual,1,2010-09-27 16:12:00,10468.8,14063.0,United Kingdom,10468.8
74356,496115,M,Manual,1,2010-01-29 11:04:00,8985.6,17949.0,United Kingdom,8985.6
698843,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,8142.75


The high value prices are almost all manual orders.  There are a few postage transactions and amazon fees as well. I'll leave them for now.


**Adjustments**<a id='adjust'></a>

In [63]:
df[df.StockCode=='ADJUST']

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
70975,495732,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:20:00,96.46,NaN,EIRE,96.46
70976,495733,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:21:00,68.34,14911.0,EIRE,68.34
70977,495735,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:22:00,201.56,12745.0,EIRE,201.56
70978,495734,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:22:00,205.82,14911.0,EIRE,205.82
70980,495736,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:23:00,21.0,12606.0,Spain,21.0
70985,495742,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:25:00,63.24,12404.0,Finland,63.24
71022,495745,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:26:00,56.73,12466.0,France,56.73
71023,495748,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:26:00,117.72,16291.0,Portugal,117.72
71033,495747,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:26:00,129.23,12555.0,Spain,129.23
71034,495750,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:27:00,132.79,14156.0,EIRE,132.79


In [64]:
df[df.Invoice==495798]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
71077,495798,ADJUST,Adjustment by john on 26/01/2010 17,1,2010-01-26 17:25:00,5117.03,NaN,United Kingdom,5117.03


In [65]:
df[df.Invoice==495750]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
71034,495750,ADJUST,Adjustment by john on 26/01/2010 16,1,2010-01-26 16:27:00,132.79,14156.0,EIRE,132.79


In [66]:
df[df.Invoice==495798]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
71077,495798,ADJUST,Adjustment by john on 26/01/2010 17,1,2010-01-26 17:25:00,5117.03,NaN,United Kingdom,5117.03


In [67]:
df[df.StockCode=='ADJUST'].shape

(36, 9)

There are only 36 adjustment transactions, all with quantity 1

### Unique Values <a id='unique'></a>

In [70]:
df.nunique()

Invoice        40076
StockCode       4916
Description     5398
Quantity         520
InvoiceDate    37212
Price           2274
Customer ID     5878
Country           43
Sales           7098
dtype: int64

**StockCode**<a id='stockcode'></a>

In [72]:
# top 20 stock codes
round(df.StockCode.value_counts(normalize=True).head(20)*100,4)

StockCode
85123A   0.5459
85099B   0.3959
22423    0.3899
21212    0.3158
20725    0.3052
84879     0.282
47566     0.263
21232    0.2453
22383    0.2407
20727    0.2385
22197    0.2348
21931    0.2302
22386    0.2229
22469    0.2201
22411    0.2183
84991    0.2167
22382    0.2141
21080    0.2124
22086     0.211
22384    0.2103
Name: proportion, dtype: float64

In [73]:
# top 20 stock codes
df.StockCode.value_counts().head(20)

StockCode
85123A    5686
85099B    4124
22423     4061
21212     3290
20725     3179
84879     2938
47566     2740
21232     2555
22383     2507
20727     2484
22197     2446
21931     2398
22386     2322
22469     2293
22411     2274
84991     2257
22382     2230
21080     2212
22086     2198
22384     2191
Name: count, dtype: int64

In [74]:
df.StockCode.isna().sum()

0

There are 4916 unique product codes and the top stock code accounts for only 0.54% of products sold

**Description**<a id='description'></a>

In [77]:
# top 20 stock codes
df.Description.value_counts().head(20)

Description
WHITE HANGING HEART T-LIGHT HOLDER    5778
REGENCY CAKESTAND 3 TIER              4061
JUMBO BAG RED RETROSPOT               3391
ASSORTED COLOUR BIRD ORNAMENT         2938
PARTY BUNTING                         2740
LUNCH BAG  BLACK SKULL.               2484
STRAWBERRY CERAMIC TRINKET BOX        2427
JUMBO STORAGE BAG SUKI                2398
HEART OF WICKER SMALL                 2293
JUMBO SHOPPER VINTAGE RED PAISLEY     2274
60 TEATIME FAIRY CAKE CASES           2257
PAPER CHAIN KIT 50'S CHRISTMAS        2198
REX CASH+CARRY JUMBO SHOPPER          2189
LUNCH BAG SPACEBOY DESIGN             2185
HOME BUILDING BLOCK WORD              2172
WOODEN FRAME ANTIQUE WHITE            2151
LUNCH BAG CARS BLUE                   2141
NATURAL SLATE HEART CHALKBOARD        2119
WOODEN PICTURE FRAME WHITE FINISH     2085
HEART OF WICKER LARGE                 2085
Name: count, dtype: int64

In [78]:
df.Description.isna().sum()

0

**StockCode with multiple Descriptions**<a id='mismatch'></a><br>
The top occurances for StockCode and Description do not all match.  There are 648 StockCodes that have multiple Descriptions.  I will leave the descriptions for reference but will use StockCode as the feature.

In [80]:
# Group by StockCode and check for multiple unique Descriptions
mismatched = df.groupby('StockCode')['Description'].nunique()

# Filter StockCodes with more than one unique Description
mismatched_codes = mismatched[mismatched > 1].index

mismatched[mismatched>1].sort_values(ascending=False)

StockCode
22346           4
23236           4
22384           4
22344           4
20685           4
               ..
22451           2
22452           2
22453           2
22454           2
BANK CHARGES    2
Name: Description, Length: 648, dtype: int64

In [81]:
# mismatched example
df[df.StockCode == 22346]['Description'].unique()

array(['PARTY PIZZA DISH GREEN+WHITE SPOT ',
       'PARTY PIZZA DISH GREEN WHITE SPOT ',
       'PARTY PIZZA DISH GREEN RETROSPOT',
       'PARTY PIZZA DISH GREEN POLKADOT'], dtype=object)

In [82]:
#matched example
df[df.StockCode==22423]['Description'].unique()

array(['REGENCY CAKESTAND 3 TIER'], dtype=object)

**Customer ID**<a id='cust_id'></a>


In [84]:
df['Customer ID'].nunique()

5878

In [85]:
#top 10 customers by total sales
df.groupby('Customer ID')['Sales'].sum().sort_values(ascending=False).head(10)

Customer ID
18102.0   608821.65
14646.0   528602.52
14156.0   313946.37
14911.0   295972.63
17450.0   246973.09
13694.0   196482.81
17511.0   175603.55
16446.0    168472.5
16684.0   147142.77
12415.0   144458.37
Name: Sales, dtype: float64

In [86]:
#top 10 customers by number of puchases

df.groupby('Customer ID')['Invoice'].count().sort_values(ascending=False).head(10)


Customer ID
17841.0    12890
14911.0    11245
12748.0     7228
14606.0     6566
14096.0     5111
15311.0     4434
14156.0     4048
14646.0     3849
13089.0     3362
16549.0     3250
Name: Invoice, dtype: int64

**Country**<a id='country'></a>

In [88]:
df['Country'].nunique()

43

In [89]:
df.Country.unique()

array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Denmark', 'Netherlands', 'Poland',
       'Channel Islands', 'Spain', 'Cyprus', 'Greece', 'Norway',
       'Austria', 'Sweden', 'United Arab Emirates', 'Finland', 'Italy',
       'Switzerland', 'Japan', 'Unspecified', 'Nigeria', 'Malta',
       'Bahrain', 'RSA', 'Bermuda', 'Hong Kong', 'Singapore', 'Thailand',
       'Israel', 'Lithuania', 'West Indies', 'Lebanon', 'Korea', 'Brazil',
       'Canada', 'Iceland', 'Saudi Arabia', 'Czech Republic',
       'European Community'], dtype=object)

In [90]:
#total number of purchases from each country
df.groupby('Country')['Invoice'].nunique().sort_values(ascending=False)

Country
United Kingdom          36534
Germany                   789
EIRE                      626
France                    622
Netherlands               228
Spain                     154
Belgium                   149
Sweden                    105
Australia                  95
Portugal                   95
Switzerland                93
Italy                      65
Finland                    57
Channel Islands            55
Norway                     45
Austria                    45
Denmark                    43
Cyprus                     37
Japan                      33
Poland                     28
Unspecified                24
USA                        20
Greece                     18
United Arab Emirates       16
Hong Kong                  15
Singapore                  11
Israel                     10
Bahrain                    10
Malta                       9
Canada                      8
Iceland                     8
Lithuania                   6
RSA                         4
Eu

### Missing Values <a id='missing'></a>
* Customer Id is missing for ~23% of entries

In [92]:
# missing values
df.isna().sum()

Invoice             0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
Price               0
Customer ID    236120
Country             0
Sales               0
dtype: int64

In [93]:
# Missing values - Percentage
df.isna().sum()/len(df)*100

Invoice                      0.0
StockCode                    0.0
Description                  0.0
Quantity                     0.0
InvoiceDate                  0.0
Price                        0.0
Customer ID   22.667469224868935
Country                      0.0
Sales                        0.0
dtype: float64

# Section 3: Export and Follow-Up <a id='explore'></a>

**Follow up** <a id='explore'></a>

* need to follow up and explore the missing 'Customer ID' to see how it will impact customer segmentation results

**Export**<a id='export'></a>

In [ ]:
#import file path
file_path

In [ ]:
# save file path
save_path = '../Data/cleaned_data.csv'

In [ ]:
# save cleaned df as .csv
df.to_csv(save_path, index=False)

**note to self:** un-comment import before submitting